In [1]:
import sys
import os

sys.path.append(os.path.join(os.getcwd(), '..') )

import pickle

import pandas as pd

from models.distance import DistanceModel
from predictor.predictor import Predictor, Formatter
from utils.utils import TextProcessor

pd.set_option('display.max_columns', 100, 'display.width', 1024)

DATA_PATH, DUMP_PATH = './data/', './dump/'

Using TensorFlow backend.


In [2]:
tp = TextProcessor()

model = DistanceModel(que_dim=34 - 2 + 8 - 2,
                      que_input_embs=[102, 42], que_output_embs=[2, 2],
                      pro_dim=42 - 2,
                      pro_input_embs=[102, 102, 42], pro_output_embs=[2, 2, 2],
                      inter_dim=20, output_dim=10)
model.load_weights(os.path.join(DUMP_PATH, 'model.h5'))

with open(os.path.join(DUMP_PATH, 'dump.pkl'), 'rb') as file:
    d = pickle.load(file) 
    que_data = d['que_data']
    stu_data = d['stu_data']
    pro_data = d['pro_data']
    que_proc = d['que_proc']
    pro_proc = d['pro_proc']
    que_to_stu = d['que_to_stu']
    pos_pairs = d['pos_pairs']

pred = Predictor(model, que_data, stu_data, pro_data, que_proc, pro_proc, que_to_stu, pos_pairs)
formatter = Formatter(DATA_PATH)

In [3]:
que_dict = {
    'questions_id': ['0'],
    'questions_author_id': ['02946e467bab4fd794e42f9670cb4279'],
    'questions_date_added': ['2017-07-29 13:30:50 UTC+0000'],
    'questions_title': [
        'I want to study law but not sure what subjects need to be taken,so need some advice on that.'],
    'questions_body': ['#law-practice #lawyer #career-details'],
    'questions_tags': ['lawyer law-practice career-details']
}

que_df, que_tags = Formatter.convert_que_dict(que_dict)

In [4]:
tmp = pred.find_pros_by_que(que_df, que_tags)
formatter.get_pro(tmp)

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined,tags_tag_name,id,match_id,match_score
1,b37293079e8841598252db357342d803,"Irvine, California",Legal Services,Paralegal,2016-03-10 21:43:07 UTC+0000,NaN,0,b37293079e8841598252db357342d803,0.8840
2,9d45a07740cf4dfeab18a09ec0b5e39b,"Atlanta, Georgia",Public Policy,Policy Analyst | Research & Analysis | Public ...,2016-03-30 15:14:27 UTC+0000,police public-policy,0,9d45a07740cf4dfeab18a09ec0b5e39b,0.8662
6,a88386e735bb48c1915c165b757decec,"Atlanta, Georgia",Law Practice,"Employment Litigation Attorney at Duguay Law, LLC",2017-04-10 19:50:24 UTC+0000,law-practice,0,a88386e735bb48c1915c165b757decec,0.8648
3,9a85abb0d2274032912473dc5ddbf5ee,"Plano, Texas",Law Practice,Law Clerk at U.S. Securities and Exchange Comm...,2017-01-28 00:37:50 UTC+0000,law-practice,0,9a85abb0d2274032912473dc5ddbf5ee,0.8618
4,364e8c67ef91428e8baf9e06ad1cac52,"Bedford, Texas",Law Practice,"Law Clerk, The Corbett Law Firm",2017-03-06 20:15:41 UTC+0000,law-practice,0,364e8c67ef91428e8baf9e06ad1cac52,0.8603
5,de0e9255e1c940e9b84f0542d74ee620,"Chicago, Illinois",Law Practice,Associate Attorney at Cray Huber Horstman Heil...,2017-03-15 19:59:29 UTC+0000,law-practice,0,de0e9255e1c940e9b84f0542d74ee620,0.8601
8,d2ee33da614a43b999ced075a958f295,Greater Los Angeles Area,Legal Services,Director of Economic Justice Initiatives at Fr...,2018-08-28 15:38:19 UTC+0000,legal-services,0,d2ee33da614a43b999ced075a958f295,0.8552
7,b2bff469d9ba480daa38bf5823439d82,"Chicago, Illinois",Law Practice,Attorney,2017-07-05 17:05:49 UTC+0000,law-practice,0,b2bff469d9ba480daa38bf5823439d82,0.8530
9,8cb4f2b09b814de38fa7ead97894357d,"Fort Worth, Texas",Legal Services,Teacher Assistant at Elementary Schools,2018-11-14 14:50:50 UTC+0000,legal-services,0,8cb4f2b09b814de38fa7ead97894357d,0.8427
0,bbfe5602a2884e3daf1f9e41f4cb84d0,"Chicago, Illinois",Legal Services,Singer/ Musician at self,2016-01-27 22:33:40 UTC+0000,law-services,0,bbfe5602a2884e3daf1f9e41f4cb84d0,0.8423


In [5]:
ans_df = pd.read_csv(os.path.join(DATA_PATH, 'answers.csv'))
que_df = pd.read_csv(os.path.join(DATA_PATH, 'questions.csv'))
que_df['questions_title'] = que_df['questions_title'].apply(tp.process)
que_df['questions_body'] = que_df['questions_body'].apply(tp.process)
ans_df['answers_body'] = ans_df['answers_body'].apply(tp.process)
que_df['questions_whole'] = que_df['questions_title'] + ' ' + que_df['questions_body']

pro_dict = {'professionals_id': ['eae09bbc30e34f008e10d5aa70d521b2'],
            'professionals_location': ['Narberth, Pennsylvania'],                
            'professionals_industry': ['Veterinary'],
            'professionals_headline': ['Veterinarian'],
            'professionals_date_joined': ['2017-09-26 18:10:23 UTC+0000'],
            'professionals_subscribed_tags': ['veterinary animal']
           }

pro_df, pro_tags = Formatter.convert_pro_dict(pro_dict)

tmp = pred.find_ques_by_pro(pro_df, que_df, ans_df, pro_tags)
formatter.get_que(tmp)

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body,tags_tag_name,id,match_id,match_score
3,52492e4bebd04d419830caa76c03d24f,c8cf9119878046f6b44839a86a0b20f8,2017-11-16 20:47:24 UTC+0000,To be a veterinarian how many years do you hav...,I want to learn all I can about to be a #veter...,veterinarian veterinary animals animal-health,eae09bbc30e34f008e10d5aa70d521b2,52492e4bebd04d419830caa76c03d24f,0.8105
6,e7696810f8aa4654bb768dd18a34bceb,f7e6dcf9a59f4486a37a71bb7e1bfacd,2016-10-19 00:04:21 UTC+0000,If I want to medically assist exotic animals i...,"This is what I want to do when I am older, so ...",veterinarian veterinary veterinary-medicine ma...,eae09bbc30e34f008e10d5aa70d521b2,e7696810f8aa4654bb768dd18a34bceb,0.7914
1,58389084774e4dfbbd2d8fc9dbc28c01,8c32713af27d4a48b5b744b59570d174,2018-04-11 14:30:53 UTC+0000,What is the hardest or most difficult thing to...,#animal-science #veterinarian #veterinary #ani...,veterinarian veterinary animals animal-science,eae09bbc30e34f008e10d5aa70d521b2,58389084774e4dfbbd2d8fc9dbc28c01,0.7874
8,daeac51d0c6142b892b602ab8ea0acb6,27ceb9082f9a4fe5a8d596cc7061169f,2016-05-19 19:03:47 UTC+0000,I hope to become a Zoo Veterinarian. Do you ha...,I will be majoring in Biology this fall. I'm i...,science biology veterinarian veterinary veteri...,eae09bbc30e34f008e10d5aa70d521b2,daeac51d0c6142b892b602ab8ea0acb6,0.7832
7,3ea1f40ee8f24cd8bc9e293e779a9e0b,3400982710e745f1a5d17d851bb3fbab,2016-05-10 13:25:11 UTC+0000,How long does it take to become a veterinarian...,I love animals and I want to be able to work w...,animal-health veterinary-technician animal-wor...,eae09bbc30e34f008e10d5aa70d521b2,3ea1f40ee8f24cd8bc9e293e779a9e0b,0.7720
0,924340cb1b414fa9a8cf843cf7715816,3400982710e745f1a5d17d851bb3fbab,2016-05-10 13:39:35 UTC+0000,"If you want to be a veterinarian tech, do yuo ...",I'm looking into this career and want to know ...,veterinarian animal-health veterinary-technici...,eae09bbc30e34f008e10d5aa70d521b2,924340cb1b414fa9a8cf843cf7715816,0.7635
5,50cbbff1193345cc944e373ed91febb6,16ce9a543e1345a393dfa8c567062781,2016-10-19 17:07:33 UTC+0000,How long after graduating from college did it ...,my future plans includes attending college and...,veterinary veterinary-medicine animals animal-...,eae09bbc30e34f008e10d5aa70d521b2,50cbbff1193345cc944e373ed91febb6,0.7604
9,36ec2fb6a1b4423d88b24bdd5657b7c2,d1c3af9aaf4e4644831652f3bb1becd0,2014-12-12 15:33:37 UTC+0000,To become a veterinarian what classes should i...,"i would like to become a veterinarian, and wou...",doctor veterinarian veterinary-medicine animal...,eae09bbc30e34f008e10d5aa70d521b2,36ec2fb6a1b4423d88b24bdd5657b7c2,0.7581
2,148f7919868041d99bfefba0694c8076,f422daffa584490da6b121ba73db981c,2018-08-18 14:13:08 UTC+0000,"What collage do you have to go to,to be a vet?",#veterinary-medicine #animal-health #veterinar...,veterinarian veterinary-medicine animals anima...,eae09bbc30e34f008e10d5aa70d521b2,148f7919868041d99bfefba0694c8076,0.7569
4,a59940dcda2149ca85007c3c6b88aa42,74653512ba5d481984ad7e47e2629a0a,2018-05-09 18:25:29 UTC+0000,How do I ensure my place in a veterinary program?,I am currently a college student trying to kno...,veterinarian veterinary animals animal-science,eae09bbc30e34f008e10d5aa70d521b2,a59940dcda2149ca85007c3c6b88aa42,0.7563


In [6]:
tmp = pred.find_pros_by_pro(pro_df, que_df, ans_df, pro_tags)
formatter.get_pro(tmp)

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined,tags_tag_name,id,match_id,match_score
3,eae09bbc30e34f008e10d5aa70d521b2,"Narberth, Pennsylvania",Veterinary,Veterinarian,2017-09-26 18:10:23 UTC+0000,veterinary #animal,eae09bbc30e34f008e10d5aa70d521b2,eae09bbc30e34f008e10d5aa70d521b2,0.9607
8,8db2be42aa36485a850e455a15dbe638,"Manhattan, Kansas",Veterinary,Student at Kansas State University,2018-02-20 18:07:24 UTC+0000,veterinary animal-health animalscience animalm...,eae09bbc30e34f008e10d5aa70d521b2,8db2be42aa36485a850e455a15dbe638,0.8112
7,e01f8be34c6042f18c31955950bde07f,"Flagstaff, Arizona Area",Veterinary,Future wildlife rehabilitator,2018-01-05 19:33:24 UTC+0000,veterinary,eae09bbc30e34f008e10d5aa70d521b2,e01f8be34c6042f18c31955950bde07f,0.7952
5,8e33eae55abe425589eaf50d904e7be8,"Boston, Massachusetts",Veterinary,"Director, Translational In Vivo Models Global ...",2017-11-11 03:18:02 UTC+0000,veterinary,eae09bbc30e34f008e10d5aa70d521b2,8e33eae55abe425589eaf50d904e7be8,0.7873
4,502818ba95374c43bd2c119ef0ca0d55,"Mohammadia, Algiers Province, Algeria",Veterinary,GBD collaborator (Global burden diseases),2017-10-22 16:28:24 UTC+0000,veterinary,eae09bbc30e34f008e10d5aa70d521b2,502818ba95374c43bd2c119ef0ca0d55,0.7817
0,e40c73d6e6bb43b6b34eb3f4329a90ef,"Franklin, New Jersey",Veterinary,CEO at Veterinary Bioscience Institute,2015-12-01 15:25:55 UTC+0000,veterinary,eae09bbc30e34f008e10d5aa70d521b2,e40c73d6e6bb43b6b34eb3f4329a90ef,0.7627
2,cb68900031664fb68d4ce41d4013747d,"Austin, Texas",Veterinary,Veterinarian at Westgate Pet and Bird Hospital,2017-01-25 19:08:38 UTC+0000,veterinary,eae09bbc30e34f008e10d5aa70d521b2,cb68900031664fb68d4ce41d4013747d,0.7538
1,573f1a3fe6b7413c9db2a0ba75d9caae,"College Station, Texas",Veterinary,Lecturer at Texas A&M University,2016-01-09 14:58:25 UTC+0000,veterinary,eae09bbc30e34f008e10d5aa70d521b2,573f1a3fe6b7413c9db2a0ba75d9caae,0.7518
6,a0b49a98c003432383bb096ecb26e6b2,"Tulsa, Oklahoma Area",Veterinary,Registered Veterinary Technician at Oklahoma V...,2017-12-15 19:19:46 UTC+0000,veterinary animals animal-health veterinary-te...,eae09bbc30e34f008e10d5aa70d521b2,a0b49a98c003432383bb096ecb26e6b2,0.7501
9,8574d1ef305e4407ade4e5ae3fea2133,Pakistan,Veterinary,veterinarian,2018-11-03 02:46:03 UTC+0000,veterinary,eae09bbc30e34f008e10d5aa70d521b2,8574d1ef305e4407ade4e5ae3fea2133,0.7497
